In [1]:
!nvidia-smi

Fri Jun  3 15:01:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 70.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create inputs for Transformers

In [31]:
import create_dataloader
from concat_four_last_layers import *
import pandas as pd
import torch
import os
from os.path import join
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoConfig,get_linear_schedule_with_warmup

In [2]:
pre_dataset_directory="/content/drive/MyDrive/en-vi-nli/pre_dataset"

In [3]:
train=pd.read_csv(f'{pre_dataset_directory}/train/train_60_100k.csv')
valid=pd.read_csv(f'{pre_dataset_directory}/valid/valid.csv')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_batch_size=16
train_premises=train['norm_premise'].tolist()
train_hypothesises=train['norm_hypothesis'].tolist()
train_labels=train['label'].tolist()

In [6]:
valid_batch_size=16
valid_premises=valid['norm_premise'].tolist()
valid_hypothesises=valid['norm_hypothesis'].tolist()
valid_labels=valid['label'].tolist()

In [7]:
xlm_name='xlm-roberta-base'
xlm_tokenizer=AutoTokenizer.from_pretrained(xlm_name)

In [8]:
rembert_name='google/rembert'
rembert_tokenizer=AutoTokenizer.from_pretrained(rembert_name)

In [33]:
max_length=72

In [34]:
xlm_train_dataloader=create_dataloader.create_dataloader(xlm_tokenizer,train_premises,train_hypothesises,train_labels,'roberta',max_length,train_batch_size,device=device,shuffle=False)

In [23]:
xlm_valid_dataloader=create_dataloader.create_dataloader(xlm_tokenizer,valid_premises,valid_hypothesises,valid_labels,'roberta',max_length,train_batch_size,device=device,shuffle=False)

  0%|          | 0/4980 [00:00<?, ?it/s]

In [11]:
rembert_train_dataloader=create_dataloader.create_dataloader(rembert_tokenizer,train_premises,train_hypothesises,train_labels,'roberta',max_length,train_batch_size,device=device,shuffle=False)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [24]:
rembert_valid_dataloader=create_dataloader.create_dataloader(rembert_tokenizer,valid_premises,valid_hypothesises,valid_labels,'roberta',max_length,train_batch_size,device=device,shuffle=False)

  0%|          | 0/4980 [00:00<?, ?it/s]

In [12]:
labels=[0,1,2]

In [13]:
epochs=30
init_lr=1e-5
eps =1e-8

In [14]:
num_labels=len(labels)
num_labels

3

In [15]:
classifier_dropout=0.1

In [16]:
rembert_config=AutoConfig.from_pretrained(rembert_name)
rembert_config.classifier_dropout=classifier_dropout
rembert_config.num_labels=num_labels
rembert_config

RemBertConfig {
  "_name_or_path": "google/rembert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 312,
  "classifier_dropout": 0.1,
  "classifier_dropout_prob": 0.1,
  "embedding_dropout_prob": 0,
  "embedding_size": 256,
  "eos_token_id": 313,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 1152,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "input_embedding_size": 256,
  "intermediate_size": 4608,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "rembert",
  "num_attention_heads": 18,
  "num_hidden_layers": 32,
  "output_embedding_size": 1664,
  "pad_token_id": 0,
  "tie_word_embeddings": false,
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250300
}

In [17]:
xlm_config=AutoConfig.from_pretrained(xlm_name)
xlm_config.classifier_dropout=classifier_dropout
xlm_config.num_labels=num_labels
xlm_config

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [18]:
model_directory='/content/drive/MyDrive/en-vi-nli/ensemble'
os.makedirs(model_directory,exist_ok=True)

In [19]:
rembert_config.save_pretrained(join(model_directory,'rember'))

In [20]:
xlm_config.save_pretrained(join(model_directory,'xlm-roberta-base'))

In [21]:
xlm_model=XLMRoBERTa4LastLayersForClassification.from_pretrained(xlm_name,config=xlm_config).to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRoBERTa4LastLayersForClassification: ['lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRoBERTa4LastLayersForClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRoBERTa4LastLayersForClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRoBERTa4LastLayersForClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['clas

In [22]:
rembert=RemBert4LastLayersForClassification.from_pretrained(rembert_name,config=rembert_config).to(device)

Downloading:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

Some weights of the model checkpoint at google/rembert were not used when initializing RemBert4LastLayersForClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RemBert4LastLayersForClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RemBert4LastLayersForClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RemBert4LastLayersForClassification were not initialized from the model checkpoint at google/rembert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
import gc
gc.collect()

326

In [28]:
xlm_optimizer=torch.optim.AdamW(xlm_model.parameters(),lr=init_lr,weight_decay =0.01,eps=eps)
rembert_optimizer=torch.optim.AdamW(rembert.parameters(),lr=init_lr,weight_decay =0.01,eps=eps)

In [29]:
def save_model(epoch,model,optimizer,scheduler,training_loss,valid_loss,train_acc,valid_acc,path,step=None):
  torch.save({
          'epoch': epoch,
          'model_state_dict': model,
          'optimizer_state_dict': optimizer,
          'scheduler_state_dict': scheduler,
          'training_loss': training_loss,
          'valid_loss': valid_loss,
          'train_acc': train_acc,
          'valid_acc': valid_acc,
          'step':step
          }, join(path,f'ckpt{epoch}.pt'))

In [35]:
len(xlm_train_dataloader),len(rembert_train_dataloader)

(6892, 6777)

In [36]:
loss_fn=nn.CrossEntropyLoss()

In [30]:
from sklearn.metrics import *


In [37]:
def run_train(model,optimizer,dataloader,lr_scheduler=None):
  model.train(True)

  loop = tqdm(dataloader)
  loss=0
  true=[]
  pred=[]
  for batch in dataloader:
    optimizer.zero_grad()
    inputs=batch['inputs']
    inputs.update({"labels":batch['labels']})
    labels=batch['labels']
    true+=labels.tolist()

    logits=model(**inputs)

    _loss=loss_fn(logits.view(-1, num_labels), labels.view(-1))
    _loss.backward()
    loss+=_loss.item()

    probs=torch.nn.functional.softmax(logits,dim=-1)
    _pred=torch.argmax(probs,dim=-1).tolist()
    pred+=_pred
    
    optimizer.step()
    if lr_scheduler!=None:
      lr_scheduler.step()
    loop.set_postfix({f'train_loss_per_batch':_loss.item()})
    loop.update()      
  acc=accuracy_score(true,pred)
  return loss/len(dataloader),acc

In [38]:
def run_valid(model,dataloader):
  model.eval()
  loss=0
  pred=[]
  true=[]
  with torch.no_grad():
    for batch in dataloader:
      inputs=batch['inputs']
      inputs.update({"labels":batch['labels']})
      labels=batch['labels']
      true+=labels.tolist()

      logits=model(**inputs)

      _loss=loss_fn(logits.view(-1, num_labels), labels.view(-1))
      loss+=_loss.item()
      
      probs=torch.nn.functional.softmax(logits,dim=-1)
      _pred=torch.argmax(probs,dim=-1).tolist()
      pred+=_pred
      

  acc=accuracy_score(true,pred)

  return loss/len(dataloader),acc

In [39]:
import gc
gc.collect()

567

In [40]:
lr_scheduler=None

In [41]:
model_directory='/content/drive/MyDrive/en-vi-nli/models/concat-4-last-layers-rembert'
os.makedirs(model_directory,exist_ok=True)
model_directory

'/content/drive/MyDrive/en-vi-nli/models/concat-4-last-layers-rembert'

In [ ]:
train_loss_per_step=[]
valid_loss_per_step=[]
train_acc_per_step=[]
valid_acc_per_step=[]
# min_loss=np.inf
best_acc=0
count_stopping=0
for epoch in range(epochs):

  train_loss,train_acc=run_train(rembert,rembert_optimizer,rembert_train_dataloader,lr_scheduler)
  
  valid_loss,valid_acc=run_valid(rembert,rembert_valid_dataloader)

  train_loss_per_step.append(train_loss)
  valid_loss_per_step.append(valid_loss)

  train_acc_per_step.append(train_acc)
  valid_acc_per_step.append(valid_acc)
  
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss} train_acc: {train_acc} valid_acc: {valid_acc}')
  # if valid_loss<min_loss:
  if best_acc<valid_acc:
    # min_loss=valid_loss
    best_acc=valid_acc
    save_model(epoch,rembert.state_dict(),rembert_optimizer.state_dict(),lr_scheduler,train_loss,valid_loss,train_acc,valid_acc,model_directory)
    count_stopping=0
  else:
    count_stopping+=1
  
  if count_stopping>5:
    break


  0%|          | 0/6777 [00:00<?, ?it/s]